In [22]:
import pandas as pd
import utils as u

# URL 1: Alta de usuarios y usos por día del servicio público de bicicleta eléctrica

### Alta Usuarios

In [23]:
usuario_abonos_anual = pd.read_excel('./data/Datos Originales/bicis_usuarios_abonos_202210.xlsx', sheet_name='Abono anual activos')
usuario_abonos_ocacional = pd.read_excel('./data/Datos Originales/bicis_usuarios_abonos_202210.xlsx', sheet_name='Abono ocasional datos')

In [24]:
#Usuarios Abonos Anual
usuario_abonos_anual.dropna(axis=0, how='all', inplace=True)
usuario_abonos_anual.dropna(axis=1, how='all', inplace=True)

#Usuarios Abonos Ocacional
usuario_abonos_ocacional.dropna(axis=0, how='all', inplace=True)
usuario_abonos_ocacional.dropna(axis=1, how='all', inplace=True)

In [25]:
usuario_abonos_anual_2022 = usuario_abonos_anual[usuario_abonos_anual['Año']==2022].reset_index(drop=True)
usuario_abonos_ocacional_2022 = usuario_abonos_ocacional[usuario_abonos_ocacional['Año']==2022].reset_index(drop=True)

In [26]:
usuario_abonos_anual_2022 = usuario_abonos_anual_2022[['Año', 
                                                       'Mes', 
                                                       'Día', 
                                                       'Altas nuevos usuarios activos ab. anual total día', 
                                                       'Altas nuevos usuarios ab. anual total acumulado mes']]

usuario_abonos_anual_2022.rename(columns={'Altas nuevos usuarios activos ab. anual total día': 'alta_ab_anual_dia',
                                        'Altas nuevos usuarios ab. anual total acumulado mes': 'alta_ab_anual_mes',
                                        'Año':'anio',
                                        'Día':'fecha'}, inplace=True)

In [27]:
usuario_abonos_ocacional_2022 = usuario_abonos_ocacional_2022[['Año', 
                                                                'Mes', 
                                                                'Día', 
                                                                'Nuevos usuarios abono 1 día ', 
                                                                'Nuevos usuarios abono 3 días ',
                                                                'Nuevos usuarios abono 5 días ',
                                                                'Nuevos usuarios abono ocas. total',
                                                                'Usuarios abono 1 día acumulado mes',
                                                                'Usuarios abono 3 días acumulado mes',
                                                                'Usuarios abono 5 días acumulado mes',
                                                                'Usuarios abono ocasional total mes']]

usuario_abonos_ocacional_2022.rename(columns={'Nuevos usuarios abono 1 día ': 'alta_ab_1d_dia',
                                        'Nuevos usuarios abono 3 días ': 'alta_ab_3d_dia',
                                        'Nuevos usuarios abono 5 días ': 'alta_ab_5d_dia',
                                        'Nuevos usuarios abono ocas. total': 'alta_ab_ocas_dia',
                                        'Usuarios abono 1 día acumulado mes': 'alta_ab_1d_mes',
                                        'Usuarios abono 3 días acumulado mes': 'alta_ab_3d_mes',
                                        'Usuarios abono 5 días acumulado mes': 'alta_ab_5d_mes',
                                        'Usuarios abono ocasional total mes': 'alta_ab_ocas_mes',
                                        'Año':'anio',
                                        'Día':'fecha'}, inplace=True)

In [28]:
usuarios = pd.merge(usuario_abonos_anual_2022, usuario_abonos_ocacional_2022, on='fecha', )

usuarios.drop(['anio_y', 'Mes_y'], axis=1, inplace=True)

usuarios.rename(columns={'anio_x':'anio',
                         'Mes_x': 'mes'}, inplace=True)

In [29]:
usuarios.to_csv('./data/Datos Tratados/usuarios_bici_resumen_2022.csv', index=False)

### Uso por dia

In [30]:
uso_bicicletas = pd.read_excel('./data/Datos Originales/bicis_usos_acumulado_202210.xlsx', sheet_name='Tabla usos')

In [31]:
uso_bicicletas.dropna(axis=0, how='all', inplace=True)
uso_bicicletas.dropna(axis=1, how='all', inplace=True)
uso_bicicletas.drop(columns='Unnamed: 21', inplace=True)

In [32]:
uso_bicicletas_2022 = uso_bicicletas[uso_bicicletas['Año'] == 2022].reset_index(drop=True)
uso_bicicletas_2022 = uso_bicicletas_2022.iloc[:, :9]
uso_bicicletas_2022.rename(columns={'Año':'anio',
                                    'DIA':'fecha',
                                    'Mes': 'mes',
                                    'Usos bicis abono anual día': 'uso_ab_anual_dia',
                                    'Usos bicis abono ocasional día':'uso_ab_ocas_dia',
                                    'Usos bicis total día':'uso_ab_total_dia',
                                    'Usos bicis abono anual acumulado mes':'uso_ab_anual_mes',
                                    'Usos bicis abono ocasional acumulado mes':'uso_ab_ocas_mes',
                                    'Usos bicis total acumulado mes':'uso_ab_total_mes'}, inplace=True)

In [33]:
uso_bicicletas_2022.to_csv('./data/Datos Tratados/uso_bici_resumen_2022.csv', index=False)

# URL 2: BiciMAD. Datos de la situación de estaciones bicimad por día y hora

In [34]:
sit_estaciones = u.call_situacion_estaciones()

In [35]:
sit_estaciones = sit_estaciones.loc[:,['_id', 'id', 'number', 'name', 'activate', 'light', 'total_bases', 'free_bases', 'dock_bikes', 'reservations_count']]
sit_estaciones.rename(columns={'_id':'fecha_hora',
                               'id':'id_estacion',
                               'number':'cod_estacion',
                               'name':'des_estacion'}, inplace=True)

In [36]:
sit_estaciones.to_csv('./data/Datos Tratados/sit_estaciones_2022.csv', index=False)

# URL 3: BiciMAD. Datos de los itinerarios de las bicicletas eléctricas

In [37]:
intinerarios_bicicletas_2022 = u.call_uso_bicicletas()

In [38]:
localizaciones = ['geolocation_unlock', 'geolocation_lock']

for campo in localizaciones:
    subindice = campo.split('_')[1]
    lon_list = []
    lat_list = []
    for i in intinerarios_bicicletas_2022.loc[:, campo]:
        valor1 = i.split("'coordinates': [")[1]
        valor2 = valor1.replace(']}', '')
        lon, lat = valor2.split(',')
        lon_list.append(lon)
        lat_list.append(lat)
    campolon = 'lon'+'_'+ subindice
    campolat = 'lat'+'_'+ subindice
    intinerarios_bicicletas_2022[campolon] = lon_list
    intinerarios_bicicletas_2022[campolat] = lat_list

In [39]:
intinerarios_bicicletas_2022.drop(columns=['geolocation_unlock', 
                                           'geolocation_lock', 
                                           'fleet', 
                                           'address_unlock', 
                                           'locktype', 
                                           'unlocktype', 
                                           'idBike',
                                           'idTrip'], axis=1)

intinerarios_bicicletas_2022 = intinerarios_bicicletas_2022.loc[:, ['fecha', 
                                                                    'trip_minutes',
                                                                    'unlock_date',
                                                                    'lon_unlock',
                                                                    'lat_unlock',
                                                                    'station_unlock',
                                                                    'dock_unlock',
                                                                    'unlock_station_name',
                                                                    'lock_date',
                                                                    'lon_lock',
                                                                    'lat_lock',
                                                                    'station_lock',
                                                                    'dock_lock',
                                                                    'lock_station_name']]

In [40]:
intinerarios_bicicletas_2022.to_csv('./data/Datos Tratados/intinerarios_bicicletas_2022.csv', index=False)

## URL 4: BiciMAD. Bases del servicio público de bicicleta eléctrica de la ciudad de Madrid

In [41]:
mapa_bases = pd.read_csv('./data/Datos Originales/bases_bicimad.csv', sep=';')
mapa_bases['Fecha de Alta'] = mapa_bases['Fecha de Alta'].apply(lambda x: u.convertir_fecha(x))
mapa_bases.drop(['Gis_X', 'Gis_Y'], axis=1, inplace=True)
mapa_bases.sort_values(by='Fecha de Alta', inplace=True, ignore_index=True)

In [42]:
mapa_bases.to_csv('./data/Datos Tratados/mapa_bases_bicicletas.csv', index=False)

## URL 5: BiciMAD. Disponibilidad de unidades (bicicletas) del servicio público de bicicleta eléctrica

In [43]:
dispo_bicis = pd.read_csv('./data/Datos Originales/bici_disponibilidad20221031.csv', sep=';')
dispo_bicis.dropna(how='all', inplace=True)
dispo_bicis.to_csv('./data/Datos Tratados/dispo_bicis_2022.csv', index=False)

## URL 6: Afor de Personas y Bicicletas

### Bicicletas

In [44]:
aforo_bicicletas_2022 = pd.read_csv('./data/Datos Originales/Aforo Personas y Bicicletas/BICICLETAS_2022.csv', sep=';')
aforo_bicicletas_2022.drop(axis=1,
                         columns=['observaciones_direccion',
                                  'identificador',
                                  'device_id',
                                  'NÃºmero_distrito',
                                  'distrito',
                                  'hora'], inplace=True)

In [45]:
aforo_bicicletas_2022.to_csv('./data/Datos Tratados/aforo_bicicletas_2022.csv', index=False)

### Personas

In [46]:
aforo_personas_2022 = pd.read_csv('./data/Datos Originales/Aforo Personas y Bicicletas/PEATONES_2022.csv', sep=';')
aforo_personas_2022.drop(axis=1,
                         columns=['observaciones_direccion',
                                  'identificador',
                                  'device_id',
                                  'NÃºmero_distrito',
                                  'distrito',
                                  'hora'], inplace=True)

In [47]:
aforo_personas_2022.to_csv('./data/Datos Tratados/aforo_personas_2022.csv', index=False)